In [ ]:
!pip install ../input/sartorius-segmentation-mask-rcnn-tf-requirements/grpcio-1.32.0-cp37-cp37m-manylinux2014_x86_64.whl
!pip install ../input/sartorius-segmentation-mask-rcnn-tf-requirements/gast-0.3.3-py2.py3-none-any.whl
!pip install ../input/sartorius-segmentation-mask-rcnn-tf-requirements/six-1.15.0-py2.py3-none-any.whl
!pip install ../input/sartorius-segmentation-mask-rcnn-tf-requirements/tensorflow_estimator-2.4.0-py2.py3-none-any.whl
!pip install ../input/sartorius-segmentation-mask-rcnn-tf-requirements/h5py-2.10.0-cp37-cp37m-manylinux1_x86_64.whl
!pip install ../input/sartorius-segmentation-mask-rcnn-tf-requirements/typing_extensions-3.7.4.3-py3-none-any.whl
!pip install ../input/sartorius-segmentation-mask-rcnn-tf-requirements/tensorflow-2.4.0-cp37-cp37m-manylinux2010_x86_64.whl
!pip install ../input/sartorius-segmentation-mask-rcnn-tf-requirements/Keras-2.4.0-py2.py3-none-any.whl
!pip install ../input/leekunhee-mask-rcnn/Mask_RCNN-master

In [ ]:
import os
import keras
import shutil
import numpy as np
import pandas as pd
import tensorflow as tf
from mrcnn import config, utils
import matplotlib.pyplot as plt 
from numpy import zeros, asarray
from mrcnn import model as modellib
from tqdm import tqdm
import cv2

In [ ]:
class CustomConfig(config.Config):
  IMAGES_PER_GPU=1
  NUM_CLASSES=2
  NAME='CustomConfig'
  USE_MINI_MASK=False
sartorius_model_cfg=CustomConfig()
#sartorius_model_cfg.display()

In [ ]:
def rle_encode(img):
    pixels=img.flatten()
    pixels=np.concatenate([[0],pixels,[0]])
    runs=np.where(pixels[:-1] != pixels[1:])[0]+1
    runs[1::2]=runs[1::2]-runs[0::2]
    #print(runs)
    return ' '.join(str(i) for i in runs)

In [ ]:
os.makedirs('./infer_model',exist_ok=True)

In [ ]:
infer_model=modellib.MaskRCNN(mode='inference',config=sartorius_model_cfg,model_dir='./infer_model')

In [ ]:
infer_model.load_weights(filepath='../input/maskrcnnweights/mask_rcnn_customconfig_0005.h5',by_name=True)

In [ ]:
def check_overlap(msk):
    msk=msk.astype(np.bool).astype(np.uint8)
    return np.any(np.sum(msk,axis=-1)>1)

def fix_overlap(msk):
    msk=msk.astype(int)
    msk=np.pad(msk,((0,0),(0,0),(1,0)))#adding dummy background along the depth axis
    msk=np.argmax(msk,axis=-1)#removing the overlapping mask
    msk=tf.keras.utils.to_categorical(msk,num_classes=msk.shape[-1])#converting back to multichannel instances
    msk=msk[:,:,1:]#removing background mask
    msk=msk.reshape(520*704,msk.shape[-1])
    msk=msk[:,np.any(msk,axis=0)]#removing zero masks
    #msk=msk[...,np.any(msk,axis=(0,1))]
    msk=msk.reshape(520,704,msk.shape[1])
    
    return msk

In [ ]:
img_names=[]
pred_masks_annot=[]
for img in tqdm(os.listdir('../input/sartorius-cell-instance-segmentation/test')):
    img_name=[]
    pred_mask_annot=[]
    img_arr=cv2.imread('../input/sartorius-cell-instance-segmentation/test/'+img)
    if len(img_arr.shape)!=3:
        img_arr=cv2.cvtColor(img_arr,cv2.COLOR_GRAY2RGB)
    if img_arr.shape[-1]==4:
        img_arr=img_arr[:,:,:3]
    results=infer_model.detect([img_arr])[0]['masks']
    print(results.shape,img_arr.shape)
    if results.shape[-1]==0:
        img_name.append(img.split('.')[0])
        pred_mask_annot.append('')
        print('No mask Found')
    else:
        if check_overlap(results):#checking overlaps 
            print('Overlap Found')
            results=fix_overlap(results)
            
        print('Shape after removing overlaps ',results.shape)    
        for depth in range(results.shape[-1]):
            pred_mask=results[:,:,depth]
            #print(pred_mask)
            pred_mask=pred_mask.astype(np.uint8)
            
            img_name.append(img.split('.')[0])
            pred_mask_annot.append(rle_encode(pred_mask))
            
    img_names.extend(img_name)
    pred_masks_annot.extend(pred_mask_annot)

In [ ]:
final_df=pd.DataFrame({'id':img_names,'predicted':pred_masks_annot}).sort_values(by='id')

In [ ]:
final_df

In [ ]:
shutil.rmtree('./infer_model')

In [ ]:
final_df.to_csv('submission.csv',index=False)